<a href="https://colab.research.google.com/github/Nech-C/comp5300-project/blob/main/5530project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
path = "/content/drive/My Drive/GQA/"

# import lib
import os
import sys
import json
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
import pandas as pd
from tqdm import tqdm
from google.colab import drive


# This will prompt for authorization.
drive.mount('/content/drive')
# Replace 'My Drive' with the specific path where you want to save the files in your Drive if needed

# Create the directory if it doesn't exist
%cd "$path"

!git -C "/content/drive/My Drive/GQA/comp5300-project" pull

sys.path.append(os.path.join(path, "comp5300-project"))
from utils.dataset_processing import *

Mounted at /content/drive
/content/drive/My Drive/GQA
Already up to date.


In [2]:
# constants - google drive
SCRIPT_PATH = "/content/drive/MyDrive/GQA/comp5300-project"
IMAGE_ZIP = '/content/drive/MyDrive/GQA/images.zip'
QUESTIONS_ZIP = '/content/drive/MyDrive/GQA/questions.zip'
SCENE_GRAPHS_ZIP = '/content/drive/MyDrive/GQA/sceneGraphs.zip'

# constants - vm
BASE = '/content/gqa'

!cp $IMAGE_ZIP $BASE/images
!cp $QUESTIONS_ZIP $BASE/questions
!cp $SCENE_GRAPHS_ZIP $BASE/sceneGraphs
# create dirs and unzip those in BASE
!mkdir -p $BASE/images
!mkdir -p $BASE/questions
!mkdir -p $BASE/sceneGraphs
!unzip -q $QUESTIONS_ZIP -d $BASE/questions
!unzip -q $SCENE_GRAPHS_ZIP -d $BASE/sceneGraphs
!unzip -q $IMAGE_ZIP -d $BASE/images

cp: cannot create regular file '/content/gqa/images': No such file or directory
cp: cannot create regular file '/content/gqa/questions': No such file or directory
cp: cannot create regular file '/content/gqa/sceneGraphs': No such file or directory


In [ ]:
import zipfile

IMAGE_ZIP = '/content/drive/MyDrive/GQA/images.zip'

with zipfile.ZipFile(IMAGE_ZIP, 'r') as zip_file:
    num_files = len(zip_file.namelist())

print(f"Number of files in images.zip: {num_files}")

Number of files in images.zip: 148855


In [3]:
import os
import json

TRAIN_ALL_QUESTIONS = "/content/gqa/questions/train_all_questions"
IMAGES = "/content/gqa/images/images"
PROCESSED_QUESTIONS = "/content/drive/MyDrive/GQA/processed_questions"

# Create the processed_questions directory if it doesn't exist
os.makedirs(PROCESSED_QUESTIONS, exist_ok=True)

# Get a list of all the image names in the images directory
image_names = set(os.listdir(IMAGES))

# Get a list of all the JSON files in the train_all_questions directory
json_files = [f for f in os.listdir(TRAIN_ALL_QUESTIONS) if f.endswith('.json')]

for json_file in json_files:
    json_path = os.path.join(TRAIN_ALL_QUESTIONS, json_file)

    with open(json_path, 'r') as f:
        questions_data = json.load(f)

    processed_questions = {}

    for question_id, question_info in questions_data.items():
        image_id = question_info['imageId']
        image_name = image_id + '.jpg'

        # Check if the image exists in the set of image names
        if image_name in image_names:
            processed_question = {
                'imageId': image_id,
                'question': question_info['question'],
                'answer': question_info['answer'],
                'fullAnswer': question_info['fullAnswer'],
                'groups': question_info['groups']
            }
            processed_questions[question_id] = processed_question
    # Save the processed questions to a new JSON file
    processed_json_path = os.path.join(PROCESSED_QUESTIONS, json_file)
    with open(processed_json_path, 'w') as f:
        json.dump(processed_questions, f, indent=2)

print("Processing completed.")

Processing completed.


In [5]:
# create trainable .jsons
processed_json_files = [os.path.join(PROCESSED_QUESTIONS, f) for f in os.listdir(PROCESSED_QUESTIONS) if f.endswith('.json')]
trainable_save_path = "/content/drive/MyDrive/GQA/trainable_questions"
os.makedirs(trainable_save_path, exist_ok=True)

for json_file in processed_json_files:
    with open(json_file, 'r') as f:
        questions_data = json.load(f)

        # generate trainable .json
        trainable_questions = []
        for question_id, question_info in questions_data.items():
            image_name = question_info['imageId'] + '.jpg'

            trainable_question = {
              'id': question_id,
              'image': image_name,
              "conversation": [
                  {
                      "from": "human",
                      "value": question_info['question']
                  },
                  {
                      "from": "gpt",
                      "value": question_info['answer']
                  }
              ],
            }
            trainable_questions.append(trainable_question)
            # end of for

        trainable_json_path = os.path.join(
            trainable_save_path,
            os.path.splitext(os.path.basename(json_file))[0] + "_trainable.json")

        with open(trainable_json_path, 'w') as f:
            json.dump(trainable_questions, f, indent=2)

print("Processing completed.")


Processing completed.


In [16]:
!pip install deepspeed
!git lfs install
!git clone https://huggingface.co/liuhaotian/llava-v1.6-mistral-7b